In [1]:
from fedn import APIClient
import time
import uuid
import json
import matplotlib.pyplot as plt
import numpy as np
import collections

In [2]:
DISCOVER_HOST = '127.0.0.1'
DISCOVER_PORT = 8092
client = APIClient(DISCOVER_HOST, DISCOVER_PORT)

In [4]:
client.set_package('package.tgz', 'numpyhelper')
client.set_initial_model('seed.npz')
seed_model = client.get_initial_model()

In [8]:
client.list_clients()

{'count': 2,
 'result': [{'combiner': 'combiner',
   'combiner_preferred': False,
   'id': 'client2',
   'ip': '172.18.0.8',
   'last_seen': 'Mon, 08 Apr 2024 07:27:34 GMT',
   'status': 'online'},
  {'combiner': 'combiner',
   'combiner_preferred': False,
   'id': 'client1',
   'ip': '172.18.0.7',
   'last_seen': 'Mon, 08 Apr 2024 07:27:04 GMT',
   'status': 'online'}]}

In [6]:
session_id = "experiment_fedavg"

session_config_fedavg = {
                        "helper": "numpyhelper", 
                        "session_id": session_id, 
                        "aggregator": "fedavg", 
                        "model_id": seed_model['model_id'], 
                        "rounds": 2, 
                        "round_timeout": 180000, 
                        "validate": False
                        }

result_fedavg = client.start_session(**session_config_fedavg)

In [7]:
client.session_is_finished(session_id)

False

In [14]:
result_fedavg

{'message': 'Session already exists.', 'success': False}